## 1. Library

In [1]:
# import library
import pandas as pd
import json
import os
import datetime
import numpy as np

# 2. Table Format Initialization

In [2]:
# initialized table format
accounts_frm = {
    'id' : [],
    'op' : [],
    'ts' : [],
    'account_id' : [],
    'name' : [],
    'address' : [],
    'phone_number' : [],
    'email': [],
    'savings_account_id': [],
    'card_id': []
}

cards_frm = {
    'id' : [],
    'op' : [],
    'ts' : [],
    'card_id' : [],
    'card_number' : [],
    'credit_used' : [],
    'monthly_limit' : [],
    'status': []
}

savings_accounts_frm = {
    'id' : [],
    'op' : [],
    'ts' : [],
    'savings_account_id' : [],
    'balance' : [],
    'interest_rate_percent' : [],
    'status': []
}


# 3. Function

In [3]:
# initialized function

def json_to_df(drc,tb_form) :
    # list json file in directory and loop through all file 
    json_list = os.listdir(f'data/{drc}')

    for dlist in json_list :
        # open json file
        with open(f'data/{drc}/{dlist}') as data_file:    
            dta = json.load(data_file)  

        # normalize loaded json data
        ld_temp = {}
        for key, value in dta.items() :
            if type(value) is dict : # handling for dictionary type
                dtc = value
                for key, value in dtc.items() :
                    ld_temp[key] = value
            
            else :
                if key == 'ts' :    
                    ld_temp[key] = datetime.datetime.fromtimestamp(int(str(value)[:-3])) # handling for integer type, convert to timestamp type
                else :
                    ld_temp[key] = value
        
        ld_temp_len = len(ld_temp.keys()) # max count check if no keys found between account_frm & ld_temp

        # looping for account_frm & ld_temp
        for key1, value1 in tb_form.items():
            ct_nf = 0 # initialize count check
            for key2, value2 in ld_temp.items():
                if key1 == key2 :
                    if value2 == '' :
                        tb_form[key1].append('')
                    else :
                        tb_form[key1].append(value2)
                    break
                
                else :
                    ct_nf = ct_nf + 1
                        
                if ct_nf == ld_temp_len :
                    tb_form[key1].append('') #if no keys found, add '' instead null

    jdf = pd.DataFrame(tb_form)
    
    return jdf

def join_tbl(tb_acc, tb_crd, tb_sac) :

    # added id for each table to each row
    tb_acc['account_id'] = tb_acc['id'].str[:2]
    tb_crd['card_id'] = tb_crd['id'].str[:2]
    tb_sac['savings_account_id'] = tb_sac['id'].str[:3]

    # simplify accounts table for join later
    temp_acc = tb_acc[['account_id','savings_account_id','card_id']]
    acc_join = temp_acc.query('savings_account_id != "" or card_id != "" ') # exclude row without savings_account_id or card_id
  
    # simplify cards table for join later
    temp_crd = tb_crd[['card_id','ts','credit_used']]
    crd_join = temp_crd.query('credit_used != ""') # exclude row without credit_used

    # simplify savings_accounts table for join later
    temp_sac = tb_sac[['savings_account_id','ts','balance']]
    sac_join = temp_sac.query('balance != ""') # exclude row without balance

    # join accounts with savings_accounts then join with cards
    temp_join = pd.merge(acc_join, sac_join, how = 'left', on = 'savings_account_id')
    all_join = pd.merge(temp_join, crd_join, how = 'left', on = 'card_id')

    # rename column for timestamp from cards & savings_accounts
    all_join.rename(columns = {'ts_x':'balance_timestamp','ts_y':'credit_used_timestamp'}, inplace = True)

    return all_join

# 4. JSON Conversion to Dataframe

In [4]:
# json conversion to dataframe for accounts
accounts_df = json_to_df('accounts',accounts_frm)

print(accounts_df)

           id op                  ts account_id     name   address  \
0  a1globalid  c 2020-01-01 14:30:00         a1  Anthony  New York   
1  a1globalid  u 2020-01-01 15:00:00                                 
2  a1globalid  u 2020-01-01 22:00:00                                 
3  a1globalid  u 2020-01-01 23:00:00                       Jakarta   
4  a1globalid  u 2020-01-02 08:00:00                                 
5  a1globalid  u 2020-01-15 16:01:00                                 
6  a1globalid  u 2020-01-16 15:30:00                                 

  phone_number                    email savings_account_id card_id  
0     12345678     anthony@somebank.com                             
1     87654321                                                      
2                                                      sa1          
3               anthony@anotherbank.com                             
4                                                               c1  
5                        

In [5]:
# json conversion to dataframe for cards
cards_df = json_to_df('cards',cards_frm)

print(cards_df)

           id op                  ts card_id card_number credit_used  \
0  c1globalid  c 2020-01-02 08:00:00      c1    11112222           0   
1  c1globalid  u 2020-01-05 00:30:00                                   
2  c1globalid  u 2020-01-06 19:30:00                           12000   
3  c1globalid  u 2020-01-08 01:00:00                           19000   
4  c1globalid  u 2020-01-10 18:00:00                               0   
5  c1globalid  u 2020-01-15 16:00:00                                   
6  c2globalid  c 2020-01-16 15:30:00      c2    12123434           0   
7  c2globalid  u 2020-01-18 05:00:00                                   
8  c2globalid  u 2020-01-18 22:30:00                           37000   

  monthly_limit   status  
0         30000  PENDING  
1                 ACTIVE  
2                         
3                         
4                         
5                 CLOSED  
6         70000  PENDING  
7                 ACTIVE  
8                         


In [6]:
# json conversion to dataframe for savings_accounts
savings_accounts_df = json_to_df('savings_accounts',savings_accounts_frm)

print(savings_accounts_df)

            id op                  ts savings_account_id balance  \
0  sa1globalid  c 2020-01-01 22:00:00                sa1       0   
1  sa1globalid  u 2020-01-02 16:00:00                      15000   
2  sa1globalid  u 2020-01-05 00:31:00                              
3  sa1globalid  u 2020-01-10 16:30:00                      40000   
4  sa1globalid  u 2020-01-10 18:00:00                      21000   
5  sa1globalid  u 2020-01-15 16:01:00                              
6  sa1globalid  u 2020-01-18 05:01:00                              
7  sa1globalid  u 2020-01-20 14:30:00                      33000   

  interest_rate_percent  status  
0                   1.5  ACTIVE  
1                                
2                   3.0          
3                                
4                                
5                   1.5          
6                   4.0          
7                                


# 5. Dataframe Join

In [7]:
# join three table
denorm_table = join_tbl(accounts_df,cards_df,savings_accounts_df)

In [8]:
# print the result
print(denorm_table)

   account_id savings_account_id card_id   balance_timestamp balance  \
0          a1                sa1         2020-01-01 22:00:00       0   
1          a1                sa1         2020-01-02 16:00:00   15000   
2          a1                sa1         2020-01-10 16:30:00   40000   
3          a1                sa1         2020-01-10 18:00:00   21000   
4          a1                sa1         2020-01-20 14:30:00   33000   
5          a1                         c1                 NaT     NaN   
6          a1                         c1                 NaT     NaN   
7          a1                         c1                 NaT     NaN   
8          a1                         c1                 NaT     NaN   
9          a1                         c2                 NaT     NaN   
10         a1                         c2                 NaT     NaN   

   credit_used_timestamp credit_used  
0                    NaT         NaN  
1                    NaT         NaN  
2                 

### From the joined table above, we can conclude that :

#### 1. account_id a1 has 1 savings_account_id & 2 card_id (c1 has already closed on 2020-01-10 and new card, c2, start active on 2020-01-18)
#### 2. There are total of 8 transactions happening in account_id a1. The details are :
#####  a. 4 from sa1, occured in January 2 (increased by 15000), 10 (increased by 25000, decreased by 19000) , and 20 (increased by 12000)
#####  b. 3 from c1, occured in January 6 (increased by 12000), 8 (increased by 7000), and 10 (decreased by 19000)
#####  c. 1 from c2, occured in January 18 (increased by 37000)